# 2.收缩方法(shrink method)
 又称正则化（regularization)。两种常用的将回归系数往零的方向压缩的方法是岭回归和lasso。

## 2.1 背景简介：
如果数据的特征比样本点要多，即($n<p$)时，输入数据的矩阵不是满秩矩阵。就不能再使用之前的线性回归方法对参数进行求解。$X^TX$矩阵奇异，逆矩阵不存在。为了解决这一问题，统计学家引入了一些缩减方法，如：岭回归，Lasso，LAR，PCA，PCR回归以及子集选择等。本节将逐一介绍。  

## 2.2 岭回归(Ridge regression)
岭回归是一种专用于共线性数据分析的有偏估计回归方法，实际上是一种改良的最小二乘估计法，通过放弃最小二乘法的无偏性，以损失部分信息，降低精度为代价获得回归系数更符合实际，更可靠的回归方法，对病态数据的拟合要强于最小二乘法。  

### 2.2.1 一般形式：  
岭回归与最小二乘相似，通过最小化下式得到：  
$$\sum_{i=1}^n(y_i-\theta_0-\sum_{j=1}^p\theta_j x_{ij})^2+\lambda \sum_{j=1}^p\theta_j^2$$  
其中，$\lambda\ge 0$是一个调节参数，将单独确定。$\lambda \sum_{j=1}^p\theta_j^2$被称为**压缩惩罚**。  
当$\theta_1,\cdots,\theta_p$接近0时比较小，因此具有将估计值往零的方向进行压缩的作用。  

**注意**：  

1. 上式对$\theta_1,\cdots,\theta_p$施加了压缩惩罚，但未对常数$\theta_0$进行惩罚。我们是要缩减与响应变量存在关联关系的预测变量的系数但是并不缩减截距项，因为截距项用于测量当$x_{i1}=x_{i2}=\cdots=x_{ip}=0$时响应变量的均值。  
2. 最小二乘系数估计是尺度不变的，但岭回归由于系数平方和项的存在，尺度是变化的。因此，在使用岭回归之前，最好先对预测变量进行标准化。  


### 2.2.2 思想：  
这里的数据矩阵X的列在岭回归之前已经进行标准化，均值为0，方差为1。   

**注意截距$\theta_0$是排除在罚项之外的。截距的罚将使得过程依赖于Y的原点选择。我们用$\bar{y}=\sum_{i=1}^ny_i/n$  估计
$\theta_0$.其余的系数使用中心化的$x_{ij}$，通过无截距的岭回归估计得到。**


简单说来，岭回归就说在矩阵$X^TX$上加一个$\lambda I$从而使得矩阵非奇异，进而能对$X^TX+\lambda I$求逆，其中矩阵$I$是一个$p\times p$维的单位矩阵，对角线上元素全为1，其他元素全为0.而$\lambda$是一个用户定义的数值。  
在这种情况下，回归系数的计算公式将变成：  
$$\hat{\theta}=(X^TX+\lambda I)^{-1}X^Ty\qquad (*)$$


### 2.2.3 理论推导：
岭回归通过对系数大小施加惩罚，在平方误差的基础上添加了正则项，解决了普通最小二乘法的一些问题。岭系数最小化惩罚残差平方和：  
$$ \min\limits_{\theta}\dfrac{1}{2}\lVert X\theta-y \rVert _2^2+\dfrac{1}{2}\lambda \lVert \theta \rVert _2^2\qquad (**)$$
这里，$\lambda\ge0$是控制收缩量的复杂参数:$\lambda$值越大，收缩量越大，从而系数对于共线性更加稳健。

回归系数的计算公式也可由损失函数最小化求得。数学推导如下：
$$\dfrac{1}{2}\lVert X\theta-y \rVert _2^2+\dfrac{1}{2}\lambda \lVert \theta \rVert _2^2=
\dfrac{1}{2}\theta^TX^TX\theta-\theta^TX^Ty+\dfrac{1}{2}y^Ty+\dfrac{\lambda}{2}\theta^T\theta$$  

对函数求导，得$X^TX\theta-X^Ty+\lambda\theta$，令其等于0，得到参数的估计值与(*)式结果相同。  

### 思考：岭回归中的岭指的是什么？  
岭回归使用了单位矩阵乘以常量$\lambda$，观察其中的单位矩阵，可以看到1贯穿整个对角线，其余元素全为0.形象地，在0构成的平面上有一条1组成的'岭'，这就是岭的由来。
#### 注意：
通常岭回归方程的R平方值会稍低于普通回归分析，但回归系数的显著性往往明显高于普通回归，在存在共线性问题和病态数据偏多的研究中有较大的实用价值。
### 问题：岭回归要对数据标准化吗？
#### 岭回归和缩减技术需要对特征作标准化处理，使每维特征具有相同的重要性。
  当原始数据不同维度上的特征的尺度（单位）不一致时，需要标准化步骤对数据进行预处理。  
具体的做法是所有的特征都减去各自的均值并除以方差。  因为无论ridge regression和lasso，它们的损失函数求解，不中心化求出来的$\theta$都不一样，普通的线性回归就没有任何问题。  
  
在本节中，假定已进行中心化，从而输入矩阵X有p列(而不是p+1列)。

In [2]:
from sklearn import preprocessing
import numpy as np
def ridgereg(X_stand,y,lam):
    p=X_stand.shape[1]
    xTxlam=X_stand.T*X_stand+lam*np.eye(p)
    if np.linalg.det(xTxlam)== 0:
        print('This matrix is singular')
        return
    theta=xTxlam.I*(X_stand.T*y)
    return theta
def ridgetest(X,y):
    n=len(y)
    Xmat=np.mat(X)
    ymat=np.mat(y).T
    Xmean=np.mean(Xmat,axis=0)
    Xvar=np.var(Xmat,axis=0)
    Xmat=np.mat(preprocessing.scale(Xmat))
    # 使用不同的lambda，看一下效果
    numTest=10
    Wmat=np.zeros((numTest,Xmat.shape[1]))
    for i in range(numTest):
        ws=ridgereg(Xmat,ymat,np.exp(i-10))
        Wmat[i,:]=ws.T
    return Wmat
# 为什么要检查行列式是否为0:
# 当lam设定为0时，岭回归回到普通线性回归，有可能会产生错误。

In [3]:
X=[[0,0],[1,1],[2,2]]
y=[0,1,2]
w=ridgetest(X,y)
print('w is :\n',w)

w is :
 [[ 0.4082452   0.4082452 ]
 [ 0.40823989  0.40823989]
 [ 0.40822547  0.40822547]
 [ 0.40818625  0.40818625]
 [ 0.4080797   0.4080797 ]
 [ 0.40779035  0.40779035]
 [ 0.40700586  0.40700586]
 [ 0.40488859  0.40488859]
 [ 0.39924301  0.39924301]
 [ 0.38466334  0.38466334]]


/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, _DataConversionWarning)


 ### 2.2.4 岭回归的缺点  
岭回归的最终模型包含全部的P个变量。公式中的惩罚项可以将系数往零的方向进行缩减，但是不会把任何一个变量的系数确切的压缩为0(除非$\lambda=\infty$)。  
lasso 是一种近年来
常用的用于克服岭回归上述缺点的方法。

## 2.3 Lasso
Lasso(Least absolute shrinkage and selection operator)  
Lasso 是一种估计稀疏线性模型的方法.由于它倾向具有少量参数值的情况，对于给定解决方案是相关情况下，有效的减少了变量数量。 因此，Lasso及其变种是压缩感知(压缩采样)的基础。  

### 2.3.1 一般形式
lasso与岭回归具有相似的拟合形式，通过最小化下式得到：  
$$\sum_{i=1}^n(y_i-\theta_0-\sum_{j=1}^p\theta_j x_{ij})^2+\lambda \sum_{j=1}^p|\theta_j|$$  
其中，$\lambda\ge 0$是一个调节参数，将单独确定。$\lambda \sum_{j=1}^p|\theta_j|$被称为**压缩惩罚**。lasso采用$l_1$惩罚项，而不采用$l_2$惩罚项。  
与岭回归相同，当$\theta_1,\cdots,\theta_p$接近0时比较小，因此lasso也具有将估计值往零的方向进行压缩的作用。然而，当$\lambda$足够大的时候，$l_1$惩罚项具有将其中某些系数的估计值强制设定为0的作用。因此，lasso也完成了变量选择。  
### 2.3.2 求解  
LAR是求解LASSo的一种有效算法

### 2.4 岭回归和lasso的其它形式
lasso和岭回归的系数估计分别等价于求解以下问题：  
$$\min\limits_{\theta}\lVert X\theta-y \rVert _2^2\qquad
s.t. \lVert \theta \rVert \le s \qquad(1)$$
$$\min\limits_{\theta}\lVert X\theta-y \rVert _2^2\qquad
s.t. \lVert \theta \rVert _2^2\le s\qquad(2)$$ 

也就是说，对于每一个$\lambda$,都有一些s使得上式得到相同的lasso系数估计，和相同的岭回归系数估计。  
当p=2, (1)式表示在$|\theta_1|+|\theta_2|\le s$所确定的菱形区域里，lasso参数估计可以得到最小的RSS。  
(2)式表示在$\theta_1^2+\theta_2^2\le s$所确定的圆形区域里，岭回归系数估计可以得到最小的RSS。  

![rigde and lasso](24.png)
### 2.4.1 lasso的变量选择特征
**为什么lasso 可以将系数估计完全压缩为0，而岭回归不可以呢？**  
如上图，最小二乘的估计系数记为$\hat{\beta}$,菱形和圆形对应lasso和岭回归的限制区域。  

如果s足够大，那么限制条件区域将包含$\hat{\beta}$，并且此时岭回归和lasso系数估计将同最小二乘系数估计一致 (这样的s与$\lambda=0$对应)。  
显然，图中显示，最小二乘系数落在了限制区域之外，所以它不同于lasso和岭回归结果。  
以$\hat{\beta}$为中心的每一个椭圆表示了一个RSS值。也就是说，在一个给定的椭圆边界上，每个点所代表的RSS值相等。lasso和岭回归都是由其条件区域与椭圆第一次相交点所决定的。  

因为岭回归的条件区域是没有尖点的圆形，所以相交点一般不会出现在坐标轴上，所以岭回归系数估计不为零。而lasso的条件区域在每个坐标轴上都有拐点，所以椭圆经常在坐标轴上与条件区域相交，这样，其中的一个系数就会为0。  
当p>2时，岭回归的限制区域将变成超球面，lasso的将变成多面体。由于多面体存在尖角，lasso将执行变量选择。   

### 2.5 岭回归和lasso的贝叶斯解释  
对于回归，贝叶斯理论假设回归系数向量$\theta$具有先验分布 $p(\theta)=(\theta_0,\theta_1,\cdots,\theta_p)^T$。将先验分布与似然函数相乘后可以得到后验分布(与后验分布成比例)，形式如下：   

$$P(\theta|X,Y)\propto f(Y|X,\theta)p(\theta|X)=f(Y|X,\theta)p(\theta)$$  

这里的比例服从贝叶斯定理，并且上面的等式在X固定的假设下成立。  
假设普通现象模型为：  
$$ Y=\theta_0+\theta_1X_1+\cdots+\theta_pX_p+\epsilon$$  
残差独立并且服从正态分布。进一步假设 $p(\theta)=\prod_{j=0}^pg(\theta_j)$ , g是密度函数。  
岭回归和lasso的密度函数 g是以下两种特殊形式的情况：  

- 如果g是高斯分布，均值为零，标准差为$\lambda$, 那么给定数据，$\theta$后验形式最可能为岭回归得到的结果。  
- 如果g是双指数-laplace 分布，均值为零，尺度参数为$\lambda$， 那么它的后验形式最可能是lasso的结果。

 


### 证明
1. 假设 $\epsilon_i\sim N(0,\sigma^2), \theta\sim N(0,\tau^2)$,那么用最大后验推导： 
$$\arg\max_\theta L(\theta) = \ln\left(\prod_{i=1}^n\dfrac{1}{\sqrt{2\pi}\sigma}\exp(-\dfrac{1}{2\sigma^2}(y_i-\theta^Tx_i)^2)\prod_{j=1}^p
\dfrac{1}{\sqrt{2\pi}\tau}\exp(-\dfrac{1}{2\tau^2}\theta_j^2)\right)$$  
$$\arg\max_\theta L(\theta) =-\dfrac{1}{2\sigma^2}\sum_{i=1}^n(y_i-\theta^Tx_i)^2-\dfrac{1}{2\tau^2}\sum_{j=1}^p \theta_j^2-n\ln \sigma\sqrt{2\pi}-p\ln \tau\sqrt{2\pi}$$   
$$\arg\min_w f(w) = \sum_{i=1}^n(y_i-\theta^Tx_i)^2+\lambda\sum_{j=1}^d\theta_j^2$$

## 2.4 弹性网

## 2.5 SCAD

## 2.6 group lasso

# 总结
https://www.cnblogs.com/monoSLAM/p/5257589.html  
从概率论的角度：  
a、最小二乘（Least Square）的解析解可以用Gaussian分布以及最大似然估计求得  
b、Ridge回归可以用Gaussian分布和最大后验估计解释  
c、LASSO回归可以用Lapace分布和最大后验估计解释